## [Improve Accuracy of OCR using Image Preprocessing](https://medium.com/cashify-engineering/improve-accuracy-of-ocr-using-image-preprocessing-8df29ec3a033)
- OCR output highly depends on the quality of input image.    
- This is why every OCR engine provides guidelines regarding the quality of input image and its size. 
- These guidelines help OCR engine to produce accurate results.    
- Here Image Preprocessing comes into play to improve the quality of input image so that the OCR engine gives you an accurate output.
---
- OCR의 결과는 입력 이미지의 Quality에 많은 의존성을 가진다
- 이것은 모든 OCR Engine이 입력 이미지의 Quality와 이미지의 사이즈에 대한 가이드라인을 제공하는 이유이다
- 이러한 가이드라인은 OCR Engine이 정확한 결과를 생산하는 것을 도울것이다
- 이미지 전처리는 입력이미지의 Quality를 향상 시켜서 OCR Engine이 너에게 정확한 결과를 주도록 할 것 이다

In [4]:
from PIL import Image
import cv2
import numpy as np

### Scaling of image 
- Image Rescaling is important for image analysis. 
- Mostly OCR engine give an accurate output of the image which has 300 DPI. 
- DPI describes the resolution of the image or in other words, it denotes printed dots per inch.
---
- 이미지 Rescaling은 이미지 분석에 매우 중요하다
- 대부분의 OCR Engine은 이미지가 300 DPI일때 정확한 결과를 준다
- DPI는 이미지의 Resolution(해상도)를 나타내고, 다른 말로 인치당 표시되는 도트의 수를 나타낸다


In [2]:
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False,   suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename